<a href="https://colab.research.google.com/github/therealthaibinh/jupyter_notebooks/blob/master/Excess_mortality_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook variables
Hint: Use `Shift+Enter` to run a cell and automatically move on to the next cell

Documentation of csv: https://www.mortality.org/Public/STMF_DOC/STMFNote.pdf

Country codes: https://www.mortality.org/cgi-bin/hmd/DataAvailability.php

In [ ]:
# You probably want to change just this number:
nWeekWindow = 26

# Possible age columns: 'D0_14', 'D15_64', 'D65_74','D75_84', 'D85p'
lstKeepAges = ['D0_14', 'D15_64', 'D65_74','D75_84', 'D85p']
# lstKeepAges = ['D65_74','D75_84', 'D85p']


#################################################################

# Probably stay the same

strURL_mortality = "https://www.mortality.org/Public/STMF/Outputs/stmf.csv"

lstCountriesEurope = ['AUT', 'BEL', 'BGR', 'CZE', 'DNK', 'GBRTENW', 'EST',
                      'FIN', 'FRATNP', 'DEUTNP', 'HUN', 'ISL', 'ITA', 
                      'LVA', 'LTU', 'LUX', 'NLD', 'NOR', 'POL', 'PRT',
                      'GBR_SCO', 'SVK', 'ESP', 'SWE', 'CHE']

nRefYearStart = 2015
nRefYearEnd = 2019 #inclusive

strSex = 'b'


# Don't touch
lstKeepCol = ['CountryCode', 'Year', 'Week', 'Sex'] + lstKeepAges

In [ ]:
%pylab inline

import pandas as pd
from datetime import datetime


# pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)

Populating the interactive namespace from numpy and matplotlib


# Load file

In [ ]:
print("LOADING DATA ON " + str(datetime.now()))
# First loading is just to get its header
print(pd.read_csv(strURL_mortality, nrows=1, header=None).loc[0,0])

# Second loading is to load actual data
df_mortality = pd.read_csv(strURL_mortality, skiprows=2)
df_mortality.head()

LOADING DATA ON 2020-09-14 16:12:53.518909
#HMD STMF pooled file. Last modified: 2020-09-10 14:35:00 


,CountryCode,Year,Week,Sex,D0_14,D15_64,D65_74,D75_84,D85p,DTotal,R0_14,R15_64,R65_74,R75_84,R85p,RTotal,Split,SplitSex,Forecast
0,AUT,2000,1,m,7.0,183.0,212.0,249.0,163.0,814.0,0.000520,0.003513,0.037607,0.095138,0.231834,0.010925,0,0,0
1,AUT,2000,1,f,2.0,104.0,141.0,338.0,468.0,1053.0,0.000156,0.002002,0.019553,0.061442,0.224357,0.013238,0,0,0
2,AUT,2000,1,b,9.0,287.0,353.0,587.0,631.0,1867.0,0.000343,0.002759,0.027474,0.072305,0.226242,0.012120,0,0,0
3,AUT,2000,2,m,4.0,195.0,195.0,259.0,187.0,840.0,0.000297,0.003743,0.034591,0.098958,0.265969,0.011274,0,0,0
4,AUT,2000,2,f,6.0,109.0,126.0,312.0,509.0,1062.0,0.000469,0.002099,0.017473,0.056716,0.244012,0.013352,0,0,0


### We only care about keeping the total sexes rows

In [ ]:
df_mortality = df_mortality[df_mortality['Sex']==strSex]
df_mortality.head()

,CountryCode,Year,Week,Sex,D0_14,D15_64,D65_74,D75_84,D85p,DTotal,R0_14,R15_64,R65_74,R75_84,R85p,RTotal,Split,SplitSex,Forecast
2,AUT,2000,1,b,9.0,287.0,353.0,587.0,631.0,1867.0,0.000343,0.002759,0.027474,0.072305,0.226242,0.012120,0,0,0
5,AUT,2000,2,b,10.0,304.0,321.0,571.0,696.0,1902.0,0.000381,0.002922,0.024983,0.070334,0.249547,0.012347,0,0,0
8,AUT,2000,3,b,13.0,342.0,346.0,573.0,753.0,2027.0,0.000495,0.003287,0.026929,0.070581,0.269984,0.013158,0,0,0
11,AUT,2000,4,b,24.0,295.0,342.0,571.0,708.0,1940.0,0.000914,0.002836,0.026618,0.070334,0.253850,0.012593,0,0,0
14,AUT,2000,5,b,16.0,304.0,314.0,563.0,731.0,1928.0,0.000609,0.002922,0.024439,0.069349,0.262096,0.012516,0,0,0


# Explore

In [ ]:
df_mortality['CountryCode'].unique()

array(['AUT', 'BEL', 'BGR', 'CHE', 'CZE', 'DEUTNP', 'DNK', 'ESP', 'EST',
       'FIN', 'FRATNP', 'GBRTENW', 'GBR_SCO', 'GRC', 'HRV', 'HUN', 'ISL',
       'ISR', 'ITA', 'LTU', 'LUX', 'LVA', 'NLD', 'NOR', 'POL', 'PRT',
       'RUS', 'SVK', 'SVN', 'SWE', 'USA'], dtype=object)

### Which countries do _not_ have all years of reference data?

In [ ]:
lstFullCountryYearWeek = []
for strCountryCode in df_mortality['CountryCode'].unique():
    for nYear in range(nRefYearStart, nRefYearEnd+1):
        for nWeek in range(1,53):
            lstFullCountryYearWeek.append(strCountryCode + " " + str(nYear) + " " + str(nWeek))

print("No reference data for these country/year/week combos:")
set(lstFullCountryYearWeek)-set((df_mortality["CountryCode"] + " " + df_mortality["Year"].astype(str) + " " + df_mortality["Week"].astype(str)).values)

No reference data for these country/year/week combos:


{'DEUTNP 2015 1',
 'DEUTNP 2015 10',
 'DEUTNP 2015 11',
 'DEUTNP 2015 12',
 'DEUTNP 2015 13',
 'DEUTNP 2015 14',
 'DEUTNP 2015 15',
 'DEUTNP 2015 16',
 'DEUTNP 2015 17',
 'DEUTNP 2015 18',
 'DEUTNP 2015 19',
 'DEUTNP 2015 2',
 'DEUTNP 2015 20',
 'DEUTNP 2015 21',
 'DEUTNP 2015 22',
 'DEUTNP 2015 23',
 'DEUTNP 2015 24',
 'DEUTNP 2015 25',
 'DEUTNP 2015 26',
 'DEUTNP 2015 27',
 'DEUTNP 2015 28',
 'DEUTNP 2015 29',
 'DEUTNP 2015 3',
 'DEUTNP 2015 30',
 'DEUTNP 2015 31',
 'DEUTNP 2015 32',
 'DEUTNP 2015 33',
 'DEUTNP 2015 34',
 'DEUTNP 2015 35',
 'DEUTNP 2015 36',
 'DEUTNP 2015 37',
 'DEUTNP 2015 38',
 'DEUTNP 2015 39',
 'DEUTNP 2015 4',
 'DEUTNP 2015 40',
 'DEUTNP 2015 41',
 'DEUTNP 2015 42',
 'DEUTNP 2015 43',
 'DEUTNP 2015 44',
 'DEUTNP 2015 45',
 'DEUTNP 2015 46',
 'DEUTNP 2015 47',
 'DEUTNP 2015 48',
 'DEUTNP 2015 49',
 'DEUTNP 2015 5',
 'DEUTNP 2015 50',
 'DEUTNP 2015 51',
 'DEUTNP 2015 52',
 'DEUTNP 2015 6',
 'DEUTNP 2015 7',
 'DEUTNP 2015 8',
 'DEUTNP 2015 9',
 'GRC 2015 1',
 'GRC 

## What is the max number of weeks available for each country in 2020?

In [ ]:
df_mortality[(df_mortality['Year']==2020)].sort_values(by=['CountryCode','Week']).\
        drop_duplicates(subset=['CountryCode'], keep='last')[['CountryCode','Week']].\
        sort_values(by='Week')

,CountryCode,Week
82700,SVN,13
70160,POL,26
50096,ITA,26
56507,LUX,26
36623,GRC,26
13748,CZE,29
73367,PRT,29
79541,SVK,30
31043,FRATNP,30
53309,LTU,31


# Doing stuff

## Get 2020 data (`df_mortality_2020`)

In [ ]:
df_mortality_2020 = df_mortality[(df_mortality['Year']==2020)][lstKeepCol]
df_mortality_2020['SUM'] = df_mortality_2020[lstKeepAges].sum(axis='columns')
df_mortality_2020.head()

,CountryCode,Year,Week,Sex,D0_14,D15_64,D65_74,D75_84,D85p,SUM
3122,AUT,2020,1,b,1.0,219.0,221.0,481.0,687.0,1609.0
3125,AUT,2020,2,b,8.0,231.0,261.0,490.0,712.0,1702.0
3128,AUT,2020,3,b,12.0,223.0,272.0,537.0,753.0,1797.0
3131,AUT,2020,4,b,13.0,259.0,256.0,515.0,736.0,1779.0
3134,AUT,2020,5,b,9.0,226.0,277.0,591.0,844.0,1947.0


## Generate reference data (`df_mortality_reference`)
For each week, get the average value over the years that were defined above

In [ ]:
df_mortality_reference = df_mortality[(df_mortality['Year']>=nRefYearStart) &\
                                       (df_mortality['Year']<(nRefYearEnd+1))]
for strCol in lstKeepAges:
    df_mortality_reference = df_mortality_reference.\
                                merge(pd.DataFrame(df_mortality_reference[['CountryCode', 'Year', 'Week']+[strCol]].groupby(by=['CountryCode','Week'])[strCol].mean()).\
                                              reset_index().\
                                              rename(columns={strCol:strCol+'_REF'}),
                                     on=['CountryCode','Week'], how='outer')

df_mortality_reference['SUM'] = df_mortality_reference[lstKeepAges].sum(axis='columns')

df_mortality_reference = df_mortality_reference.\
                                merge(pd.DataFrame(df_mortality_reference[lstKeepCol+['SUM']].groupby(by=['CountryCode','Week'])['SUM'].mean()).\
                                          reset_index().\
                                          rename(columns={'SUM':'REF_SUM'}),
                                      on=['CountryCode','Week'], how='outer')
df_mortality_reference = df_mortality_reference[['CountryCode','Week']+[x+'_REF' for x in lstKeepAges]+['REF_SUM']].drop_duplicates().round(2)
df_mortality_reference.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,CountryCode,Week,REF
0,AUT,1,1809.0
1,AUT,2,1857.0
2,AUT,3,1786.4
3,AUT,4,1747.0
4,AUT,5,1818.6


## Jam the dataframes together

In [ ]:
df_mortality_join = df_mortality_2020[lstKeepCol+['SUM']].merge(df_mortality_reference, on=['CountryCode','Week'])
df_mortality_join['DIFF'] = df_mortality_join['SUM']-df_mortality_join['REF_SUM']
df_mortality_join['EXCESS_ONLY'] = df_mortality_join['DIFF'].clip(0, None)

df_mortality_join.head()

,CountryCode,Year,Week,Sex,D0_14,D15_64,D65_74,D75_84,D85p,SUM,REF,DIFF,EXCESS_ONLY
0,AUT,2020,1,b,1.0,219.0,221.0,481.0,687.0,1609.0,1809.0,-200.0,0.0
1,AUT,2020,2,b,8.0,231.0,261.0,490.0,712.0,1702.0,1857.0,-155.0,0.0
2,AUT,2020,3,b,12.0,223.0,272.0,537.0,753.0,1797.0,1786.4,10.6,10.6
3,AUT,2020,4,b,13.0,259.0,256.0,515.0,736.0,1779.0,1747.0,32.0,32.0
4,AUT,2020,5,b,9.0,226.0,277.0,591.0,844.0,1947.0,1818.6,128.4,128.4


## Create excess mortality up to week defined above

In [ ]:
df_excess_mortality = pd.DataFrame(df_mortality_join[df_mortality_join['Week']<=nWeekWindow].groupby(by=['CountryCode'])[['REF_SUM','EXCESS_ONLY']].sum()).reset_index()
df_excess_mortality.head()

,CountryCode,REF,EXCESS_ONLY
0,AUT,42005.4,1633.8
1,BEL,57350.0,8927.8
2,BGR,56651.4,140.0
3,CHE,34447.4,1990.6
4,CZE,57361.8,910.0


# Final comparisons

In [ ]:
print("Europe:")
s_temp = df_excess_mortality[df_excess_mortality['CountryCode'].isin(lstCountriesEurope)][['REF_SUM','EXCESS_ONLY']].sum()
print("\tReference deaths:", int(s_temp['REF_SUM']))
print("\tExcess deaths:", int(s_temp['EXCESS_ONLY']))
print("\tPercent increase:", round(s_temp['EXCESS_ONLY']*100.0/s_temp['REF_SUM'], 2))

print()

print("United States:")
s_temp = df_excess_mortality[df_excess_mortality['CountryCode'].isin(['USA'])][['REF_SUM','EXCESS_ONLY']].sum()
print("\tReference deaths:", int(s_temp['REF_SUM']))
print("\tExcess deaths:", int(s_temp['EXCESS_ONLY']))
print("\tPercent increase:", round(s_temp['EXCESS_ONLY']*100.0/s_temp['REF_SUM'], 2))

Europe:
	Reference deaths: 2492041
	Excess deaths: 246620
	Percent increase: 9.9

United States:
	Reference deaths: 1426784
	Excess deaths: 194936
	Percent increase: 13.66


In [ ]:
# Run this as a reminder of what your variables were
print(lstKeepAges)
print(nWeekWindow)

['D0_14', 'D15_64', 'D65_74', 'D75_84', 'D85p']
26


# Further exploration

### All individual countries

In [ ]:
for strCountryCode in df_excess_mortality['CountryCode'].unique():
    print(strCountryCode)
    s_temp = df_excess_mortality[df_excess_mortality['CountryCode'].isin([strCountryCode])][['REF_SUM','EXCESS_ONLY']].sum()
    print("\tReference deaths:", int(s_temp['REF_SUM']))
    print("\tExcess deaths:", int(s_temp['EXCESS_ONLY']))
    print("\tPercent increase:", round(s_temp['EXCESS_ONLY']*100.0/s_temp['REF_SUM'], 2))
    print()

AUT
	Reference deaths: 42005
	Excess deaths: 1633
	Percent increase: 3.89

BEL
	Reference deaths: 57349
	Excess deaths: 8927
	Percent increase: 15.57

BGR
	Reference deaths: 56651
	Excess deaths: 140
	Percent increase: 0.25

CHE
	Reference deaths: 34447
	Excess deaths: 1990
	Percent increase: 5.78

CZE
	Reference deaths: 57361
	Excess deaths: 909
	Percent increase: 1.59

DEUTNP
	Reference deaths: 483678
	Excess deaths: 12183
	Percent increase: 2.52

DNK
	Reference deaths: 27704
	Excess deaths: 508
	Percent increase: 1.83

ESP
	Reference deaths: 220613
	Excess deaths: 48816
	Percent increase: 22.13

EST
	Reference deaths: 8029
	Excess deaths: 211
	Percent increase: 2.63

FIN
	Reference deaths: 27517
	Excess deaths: 958
	Percent increase: 3.48

FRATNP
	Reference deaths: 304729
	Excess deaths: 29737
	Percent increase: 9.76

GBRTENW
	Reference deaths: 281501
	Excess deaths: 59657
	Percent increase: 21.19

GBR_SCO
	Reference deaths: 30068
	Excess deaths: 5005
	Percent increase: 16.65

GRC
	

In [ ]:
df_mortality_join[df_mortality_join['CountryCode']=='USA']

,CountryCode,Year,Week,Sex,D0_14,D15_64,D65_74,D75_84,D85p,SUM,REF,DIFF,EXCESS_ONLY
876,USA,2020,1,b,574.565979,14762.434021,11575.401018,14356.142857,18673.456124,59942.0,59859.6,82.4,82.4
877,USA,2020,2,b,562.378216,14527.621784,11767.437713,14594.312249,18983.250038,60435.0,60813.4,-378.4,0.0
878,USA,2020,3,b,585.883187,14283.116813,11454.210294,14205.838660,18477.951046,59007.0,59685.8,-678.8,0.0
879,USA,2020,4,b,575.436533,14148.563467,11392.447141,14129.238234,18378.314625,58624.0,58575.2,48.8,48.8
880,USA,2020,5,b,563.248770,14055.751230,11367.534272,14098.340583,18338.125145,58423.0,57935.8,487.2,487.2
881,USA,2020,6,b,607.647050,14438.352950,11444.089441,14193.286489,18461.624070,59145.0,58202.6,942.4,942.4
882,USA,2020,7,b,572.824870,14165.175130,11370.388872,14101.880939,18342.730189,58553.0,57671.2,881.8,881.8
883,USA,2020,8,b,551.061007,14173.938993,11376.098071,14108.961651,18351.940279,58562.0,57026.8,1535.2,1535.2
884,USA,2020,9,b,598.941505,14376.058495,11426.183317,14171.078803,18432.737881,59005.0,56677.4,2327.6,2327.6
885,USA,2020,10,b,513.627163,14546.372837,11485.870397,14245.104425,18529.025178,59320.0,56977.0,2343.0,2343.0
